<div class="alert alert-info">
<font size="3"><b>Комментарий</b></font>
    
Поначалу неправильно понял задачу и подготовил собственные функции для трансформации. Позже поправил, но этот код сохранил для себя.
</div>

## Этап 3: Генерация Признаков и обучение новой версии модели

### 3.1 Ручная генерация признаков

На основании проведенного EDA в части подготовки дополнительных признаков можно отметить следующее:
- более оптимальным вариантом было бы использование не фактического значения признаков площади кухни `kitchen_area` и жилой площади `living_area`, а относительное к общей площади `total_area`;
- для некоторых моделей могут быть полезны признаки нахождения на первом и последнем этажах;
- также вероятно полезным может быть плотность квартир на этаж - `flats_count` / `floors_total`. Возможно более оптимальным была бы плотность квартир на этаж в подъезде, однако в датасете отсутствуют сведения о количестве подъездов в доме;
- полезными могут быть сведения о наличии лифта на высоких этажах, однако, вероятнее всего, в датасете имеются ошибки в таких сведениях. При этом можно отметить различие средней цены для квартир до 5-го этажа включительно и квартир выше 5-го этажа, в зданиях без лифта;
- в части картографических данных было бы полезно иметь сведения о расстоянии до наиболее социально значимых объектов - станции метро, школы, парки и т.д., а также до центра города. Однако такие сведения отсутствуют в датасете;
- возможно имеются и другие сложные или скрытые зависимости между признаками, однако в виду ограниченности времени на проект такие признаки попробуем сгенерировать автоматически.

Из приведенного списка можно выделить две операции, которые можно произвести с признаками - это деление признаков друг на друга, а также сравнение с другим признаком или числом. Подготовим две функции для проведения таких операций.

In [ ]:
# Функция добавления отношения двух признаков (деление)
def ratio_feature(
    df: pd.DataFrame, 
    ratio: dict
) -> pd.DataFrame:
    """
    Необходимо передать датасет, а также словарь в качестве ключа 
    в котором указано имя нового столбца датасета, а значение - 
    кортеж из двух наименований столбцов (делимого и делителя)
    ``{'new_column_name': ('base_column', 'divider_column), ...}``.
    """
    result = df.copy()
    for new_col, cols in ratio.items():
        result[new_col] = df[cols[0]] / df[cols[1]]
    return result

# Функция добавления сравнений признака со значениями
def compare_feature(
    df: pd.DataFrame, 
    compare: dict,
    astype = "int"
) -> pd.DataFrame:
    """
    Необходимо передать датасет, а также словарь с параметрами, в качестве 
    ключа в котором имя нового столбца датасета, а значения - словарь c 
    именем столбца, типом сравнения и значением (или другим столбцом):
    ```
        {
            'new_column_name': { 
                'column': 'column_name', 
                'compare': 'less' | 'equal' | 'great', 
                'value': numeric | 'column_name', 
                # Optional 
                'filter': {
                    'column': 'column_name',
                    'invert': True | False
                }
            },
            ...
        }
    ```
    Опциональный ключ ``filter`` позволяет отфильтровать полученные результаты сравнения 
    по значениям другого столбца, ключ ``invert`` инвертирует значения этого столбца.
    """
    result = df.copy()
    for new_col, settings in compare.items():
        if isinstance(settings["value"], str):
            value = result[settings["value"]]
        else:
            value = settings["value"]
        if settings["compare"] == "equal":
            result[new_col] = df[settings["column"]] == value
        elif settings["compare"] == "less":
            result[new_col] = df[settings["column"]] < value
        elif settings["compare"] == "great":
            result[new_col] = df[settings["column"]] > value
        else:
            raise ValueError("Получено неизвестное значение, должно быть одно из 'equal', 'less', 'great'.")
        if "filter" in settings:
            fltr = settings["filter"]
            if ("invert" in fltr) and fltr["invert"]:
                result[new_col] = result[new_col] & ~result[fltr["column"]]
            else:
                result[new_col] = result[new_col] & result[fltr["column"]]
        result[new_col] = result[new_col].astype(astype)
    return result

Оценим работу функции добавления признаков, путем деления значений одного признака на значения другого.

In [ ]:
ratio_feature_params = {
    "kitchen_ratio": ("kitchen_area", "total_area"),
    "living_ratio": ("living_area", "total_area"),
    "flats_by_floor": ("flats_count", "floors_total")
}
ratio_feature(
    train[["kitchen_area", "living_area", "total_area", "flats_count", "floors_total"]],
    ratio_feature_params
).sample(10, random_state=RANDOM_STATE)

Функция отработала корректно, получены 3 дополнительных признака. 

Оценим функцию сравнения признака со значением.

In [ ]:
compare_feature_params = {
    "first_floor": {
        "column": "floor",
        "compare": "equal",
        # Первый этаж
        "value": 1 
    },
    "last_floor": {
        "column": "floor",
        "compare": "equal",
        # Последний этаж
        "value": "floors_total"
    },
    "no_elevator_high_floor": {
        "column": "floor",
            "compare": "great",
            # выше 5 этажа
            "value": 5,
            # без лифта
            "filter": {
                "column": "has_elevator",
                "invert": True
            }
    }
}
compare_feature(
    train[["floor", "floors_total", "has_elevator"]],
    compare=compare_feature_params, astype="bool"
).query("has_elevator == False").sample(20, random_state=RANDOM_STATE)

Функция также обрабатывает корректно, получены еще 3 дополнительных столбца.

### 3.2 Оборачивание всех преобразований в объекты sklearn

Для того, чтобы использовать в пайплайне собственный трансформер, можно подготовить отдельный класс, как это было реализовано при обработке выбросов, но в данном случае воспользуемся классом `FunctionTrasformer` библиотеки `scikit-learn` для преобразования функции в трансформер.

In [ ]:
ratio_feat = FunctionTransformer(ratio_feature, kw_args={"ratio": ratio_feature_params})
compare_feat = FunctionTransformer(compare_feature, kw_args={"compare": compare_feature_params})

Проверим работу трансформеров.

In [ ]:
pd.concat([
    ratio_feat.transform(train[["kitchen_area", "living_area", "total_area", "flats_count", "floors_total"]]),
    compare_feat.transform(train[["floor", "floors_total", "has_elevator"]])
], axis=1).query("has_elevator == False").sample(20, random_state=RANDOM_STATE)